In [1]:
import os
import base64
import langchain
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [2]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

from chatrag.retriever import create_retriever_from_csv, create_retrieval_chain

In [3]:
langchain.debug = True

## Data preparation

In [4]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [5]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm
)

# Vectorstore DB

In [6]:
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)

In [7]:
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [8]:
g_media_sq_retrieval_chain.run("I want to watch a movie about space battles.")

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "I want to watch a movie about space battles."
}
[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 3:chain:LLMChain] Entering Chain run with input:
{
  "query": "I want to watch a movie about space battles."
}
[llm/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 3:chain:LLMChain > 4:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": string \\ logical condition statement for filtering documents\n}\n```\n\nThe query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in

/Users/itortouch/miniconda3/envs/test_pip_github/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 3:chain:LLMChain > 4:llm:ChatOpenAI] [1.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"query\": \"space battles\",\n    \"filter\": \"NO_FILTER\"\n}\n```",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "```json\n{\n    \"query\": \"space battles\",\n    \"filter\": \"NO_FILTER\"\n}\n```",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 829,
      "completion_tokens": 22,
      "total_tokens": 851
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 3:

"1. Title: Interstellar\n   - Justification: Although not primarily focused on space battles, Interstellar offers a thrilling space adventure with high stakes and intense moments. It's highly rated and well-loved by audiences.\n   - Score: 8.1\n\n2. Title: Treasure Planet\n   - Justification: This animated film features space battles and adventures, with a unique blend of traditional pirate lore and futuristic sci-fi elements.\n   - Score: 7.2\n\n3. Title: Gravity\n   - Justification: While not featuring traditional space battles, Gravity is a high-stakes survival story set in space, with intense action sequences and a high average vote score.\n   - Score: 7.3\n\n4. Title: Independence Day\n   - Justification: Independence Day is a classic alien invasion film with several large-scale space battle scenes. It's a good choice if you're looking for action-packed sequences.\n   - Score: 6.7\n\n5. Title: Space Battleship Yamato\n   - Justification: The title itself suggests space battles, an

# Tooling

In [9]:
calc_tool = load_tools(["llm-math"], llm=OpenAI(temperature=0))

In [10]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence"""

In [11]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    ),
    calc_tool[0]
]

# Prompt templates

In [12]:
prefix = """ You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. 

For the first interactions you should collect some user configuration data. This data will restrict the movies to consider.

User Data to collect (mandatory):
    Target genre: Movie genre e.g. fiction, adventure, trhiller...
    Movie overview topic: List of keywords defining the campaign context.

After succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good as you can. To do so, you have access to the following tools:"""

In [13]:
suffix = """Begin! Your first action must ve collect user data and keep it. Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation: ... Thought: ... Action: ..."""


In [14]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": prefix,
        "suffix": suffix
    }

# Chains

In [15]:
# agent_llm_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=1000)
agent_llm_model = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)

In [16]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

# TESTS

### Run

In [17]:
langchain.debug = True

In [18]:
memory.chat_memory.messages

[]

In [19]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Hi, my name is Aitor",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Hi, my name is Aitor",
  "chat_history": [],
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System:  You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. \n\nFor the first interactions you should collect some user configuration data. This data will restrict the movies to consider.\n\nUser Data to collect (mandatory):\n    Target genre: Movie genre e.g. fiction, adventure, trhiller...\n    Movie overview topic: List of keywords 

In [20]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System:  You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. \n\nFor the first interactions you should collect some user configuration data. This data will restrict the movies to consider.\n\nUser Data to collect (mandatory):\n    Target genre: Movie genre e.g. fiction, adventure, trhiller...\n    Movie overview topic: List of keywords defining the campaign context.\n\nAfter succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good a

In [21]:
response = general_chat_agent.run(input="A thriller set in space sounds good.")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System:  You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. \n\nFor the first interactions you should collect some user configuration data. This data will restrict the movies to consider.\n\nUser Data to collect (mandatory):\n    Target genre: Movie genre e.g. fiction, adventure, trhiller...\n    Movie overview topic: List of keywords defining the campaign context.\n\nAfter succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good a

In [27]:
print(response)

Based on your preferences, I would recommend the following movies: 'Interstellar', '2001: A Space Odyssey', and 'Gravity'. These are thrilling movies set in outer space and have received high ratings from viewers. Enjoy your movie time!


In [22]:
# TODO: Current issue. Though it identifies what to do, doesn't launch the action tool. 
response = general_chat_agent.run(input="space exploration")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System:  You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. \n\nFor the first interactions you should collect some user configuration data. This data will restrict the movies to consider.\n\nUser Data to collect (mandatory):\n    Target genre: Movie genre e.g. fiction, adventure, trhiller...\n    Movie overview topic: List of keywords defining the campaign context.\n\nAfter succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good a

In [23]:
print(response)

Here are some thrilling space exploration movies you might enjoy: 
1. Interstellar: A group of explorers use a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage. 
2. 2001: A Space Odyssey: Humanity finds a mysterious object buried beneath the lunar surface and sets off to find its origins with the help of an advanced super computer. 
3. Gravity: A medical engineer and an astronaut are left alone in space after their shuttle is destroyed. 
4. Event Horizon: A group of astronauts are sent to investigate and salvage a long lost starship. 
5. Pitch Black: Marooned passengers on a remote planet soon learn that escaped convict Riddick isn't the only thing they have to fear. 
I hope you find these suggestions helpful!


In [36]:
response = general_chat_agent.run(input="search it")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search movies",
  "action_input": "thriller space exploration"
}
```

/Users/itortouch/miniconda3/envs/growwer_copilot/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='thriller space exploration' filter=None limit=None

Observation: 1. url_media: Interstellar
   - Justification: The movie "Interstellar" is a perfect match for the topic "thriller space exploration". It chronicles the adventures of a group of explorers who use a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage. It has the highest vote average of 8.1 among the listed media.
   - price_sale_dofollow: [Not provided]
   - monthly_traffic: [Not provided]
   - domaing_rating: [Not provided]

2. url_media: Event Horizon
   - Justification: The movie "Event Horizon" is also a good match for the topic. It's about a group of astronauts sent to investigate and salvage the long lost starship "Event Horizon". The ship disappeared mysteriously 7 years before on its maiden voyage and with its return comes even more mystery as the crew discover the real truth behind its disappearance and something even m